In [3]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from util import full_layer, config, mse

class IModel(object):
    def train(self, x):
        pass
    def predict(self, x):
        pass


class SingleHiddenNN(IModel):
    def __init__(self, input_shape=[128, 9216], n_hidden=100, n_y=30, obj_fcn=mse):
        '''
        input_shape: (batch_size, input dimension); tuple
        n_hidden: # latent variables; int
        n_y: # output features; int
        '''
        batch_size, n_x = input_shape
        x = tf.placeholder(shape=(None, n_x), dtype=tf.float32, name='x')
        y = tf.placeholder(shape=(None, n_y), dtype=tf.float32, name='y')
        lr = tf.placeholder(dtype=tf.float32)
        
        h1 = full_layer(x, n_hidden, 'Hidden01')
        yhat = full_layer(h1, n_y, 'Output', nonlinear=tf.identity)

        # 這邊沒啥重要的，只是叫這個物件把某些 attributes 存下來而已
        self.batch_size = batch_size
        self.lr = lr
        self.x = x
        self.y = y
        self.yhat = yhat

        # Training
        self.objective = obj_fcn(y, yhat)
        self.optimizer = tf.train.AdamOptimizer(self.lr).minimize(self.objective)

        self.sess = tf.Session(config=config)
        

    def predict(self, x):
        return self.sess.run(self.yhat, feed_dict={self.x: x})

    def train(self, train_batches, valid_set=None, lr=1e-2, n_epoch=100):        
        # 叫 graph 把 initialization 排進行程裡
        init = tf.initialize_all_variables() 
        self.sess.run(init)  # 做了這步之後，所有的 weight 才會有值

        # 把 computational graph 畫出來
        summary_writer = tf.train.SummaryWriter(
            logdir='model1',
            graph=self.sess.graph)

        loss_train_record = list() # np.zeros(n_epoch)
        loss_valid_record = list() # np.zeros(n_epoch)
        start_time = time.gmtime()

        n_batch = train_batches.n_batch
        for i in range(n_epoch):
            loss_train_sum = 0.0
            loss_valid_sum = 0.0
            for x, y in train_batches:
                # ==== Training ====
                _, loss_train = self.sess.run(
                    [self.optimizer, self.objective], 
                    feed_dict={
                        self.x: x, 
                        self.y: y,
                        self.lr: lr})
                # ==== Evaluation ====
                loss_valid = self.sess.run(
                    self.objective, 
                    feed_dict={
                        self.x: valid_set.images, 
                        self.y: valid_set.labels})
                loss_train_sum += loss_train
                loss_valid_sum += loss_valid
            # Warning: 這不是 Tensorflow 的 style
            print 'Epoch %04d, %.8f, %.8f,  %0.8f' % (
                i, loss_train_sum/n_batch, loss_valid_sum/n_batch,
                loss_train_sum/loss_valid_sum)
            loss_train_record.append(loss_train_sum)    # np.log10()
            loss_valid_record.append(loss_valid_sum)    # np.log10()

        end_time = time.gmtime()
        print time.strftime('%H:%M:%S', start_time)
        print time.strftime('%H:%M:%S', end_time)
        self._error_plot(loss_train_record, loss_valid_record)

    def _error_plot(self, trn_loss, vld_loss):
        '''
        trn_loss: training_loss_record
        vld_loss: validation_loss_record
        '''
        plt.figure()
        plt.plot(trn_loss, label='train')
        plt.plot(vld_loss, c='r', label='validation')
        plt.xlabel('mini-batch')
        plt.ylabel('loss')
        plt.yscale('log')
        # plt.ylim(1e-3, 1e-2)
        plt.legend()
        plt.savefig('imgs/model1-(fully-connected)-loss.png')


## Placeholder: 一種餵 data 進去 computaional graph 的方式
讓我們從 constructor (`__init__`) 開始講  

```python
x = tf.placeholder(shape=(None, n_x), dtype=tf.float32, name='x')
```
這裡有個新東西叫 `tf.placeholder`  
他是一個很方便的東西：
1. 它讓你在建立 computational graph 的時候，可以先用 placeholder 指定好 input 的形狀，形成一個沒有內容的 tensor，以便讓 TF 能夠處理。
2. tf.placeholder 物件可以有未知大小的維度，用 None 代表

另一個要提點的地方是：learning rate (lr) 也是用 placeholder 來存  
如此一來， learning rate 就不用在 constructor 內被訂死。  
請注意這點。  
(用 tf.Variable 存很沒意思。因為它歸根究柢並非 model parameter)


**Hint**:
1. tf.placeholder 一定要指定 data type。在 GPU 運算裡，通常使用的是 tf.float32 (請不要用 tf.float64)
2. 第一軸通常用來代表 batch size；我們會把它填成 None 其實只是為了方便。
3. 如果有使用 tf.placeholder，那麼之後在 training 的時候，就要把真正的資料餵進去(feed)。
4. 實際上 TF 有提供其他餵資料的方法。但很複雜，我在此 tutorial 中不會講到。(有興趣的可以去看 TF 的 CNN cifar10 教學
)

## 指定Objective 以及 Optimizer
我這邊為了維持 objective 的可變性，我是在 class 外面定義 objective，然後透過 constructor 傳進來的。我這邊為了符合 Nouri 的作法，將 mean square error 定義如下

In [4]:
def mse(y, yhat, name='objective_mse'):
    with tf.variable_scope(name):
        obj = tf.sub(y, yhat)   # sub: 矩陣減法
        obj = tf.square(obj)
        obj = tf.reduce_mean(obj) # 取整個 tensor 的平均
    return obj

定義好 MSE之後，接下來要怎麼 minimize 它了。  
在沒有 Theano/Tensorflow 的時代，唯一的方法就是親手去導 gradient  
但現在你唯一要做的，就是呼叫 optimizer 而已。  
Optimizer 在做的事情無非就是計算 gradient 以及更新參數。  
都是些苦工。而現在，你有 TF 幫你搞定這些苦功了！


TF 目前提供了 7 種 optimizer，從最陽春的 SGD 到 Adam 都有。
我們這邊就無腦用 Adam 囉。

## Session 與 Training
之前講過，TF把流程分成 symbolic 和 numerical 兩部分。  
當你要真正把資料餵進去的時候（也就是要 training）的時候，一切才真正進入 numerical 階段。

在開始這個階段之前，你需要 `tf.Session`物件；他負責啟動/運行我們定義好的 computational graph  
它和 training 緊密相關，所以我們直接看 `train()`  

### Initialize all Variables
首先又是一個 TF 特有的東西：  
```python
init = tf.initialize_all_variables()
self.sess.run(init)
```
之前講過：在第一個階段，所有的運作都是 symbolic。  
所以，即使**宣告**了一個變數，並且給了初始值。  
該變數也不會真的有值──直到**初始化**為止。  
而 TF 中，變數需要透過上面兩行來初始化。  
有點類似在 C++ 裡面，declaration 和 initialization 是分開的一樣。  
只是 TF 把 initialization 全部集中在一個函式裡面而已。
然後請記得叫 session 去 run 它。
不然所有的 parameters 仍然沒有真正被執行。(

### Session
(然後請先略過 summary_writer 那段)
我們直接進入 training 的部分。

```python
_, loss_train = self.sess.run(
    [self.optimizer, self.objective], 
    feed_dict={self.x: x, self.y: y, self.lr: lr})
```

`sess` 是 `tf.Session` 物件，其 `run` method 負責把 symoblic 和 numeric 兩個世界連接起來。  
這邊必須說明一下：  
Computational graph 就像一局擺好的骨牌，data (本段 code 中的 x 和 y) 就像是那第一張骨牌。  
所以第一張骨牌推下去之後，所有之前定義過的東西都可以取值了。
這邊我們要求取 `self.optimizer` 和 `self.objective` 的值。
objective 是為了讓我們了解現在 training 的情況如何 (我們想知道它有沒有在降)  
那麼 optimizer 呢? 為何要取其值? 其值代表什麼意義?  
其實它不會回傳有用的值。  
前面講過，optimizer 做兩件事：  
1. 算 gradient
2. 更新參數

所以這邊叫 Session 去 run 它，意思就是要做這兩件事。


### Feed Your Networks
另一個要說明的是 `feed_dict`  
`run` 要求我們把需要傳進去的資料都存在 `dict` 裡面（因為變數可能多、可能少，用 `dict` 最方便）。  
我這邊是用一個臨時建成的 dict 來放。

Hint:
1. 事實上，training 永遠是 ad-hoc。所以其實未必應該寫在 class 裡面。我這邊的寫法僅供參考。
2. 我這裡假設傳進來的 x 是 iterator，每 iterate 一次都會產生新的 batch。  
雖然未必要這樣寫，但把資料變成 iterator 會很方便。詳請可以參考 util.py。

## 關於 evaluation
在 training loop 中，我還放了 evaluation
說真的，這個寫法並不好。  
因為通常我們不會每個 mini-batch 都跑 eval  
而且在資料龐大的時候，這種方法更顯荒謬。  
在 TF 的 Cifar10 的官方範例中，作者是用另外一支 script 去跑 eval

## Main Script

In [5]:
# coding=utf8
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from fkp_input import load_train_set, BatchRenderer
from model1 import SingleHiddenNN

# ======== 這是 tensorflow 吃 program argument 的方式 ========
FLAGS = tf.app.flags.FLAGS  
tf.app.flags.DEFINE_float('lr', 1e-2, 'learning rate')
tf.app.flags.DEFINE_float('valid', 0.2, 'fraction of validation set')
tf.app.flags.DEFINE_integer('n_epoch', 400, 'number of epochs')
tf.app.flags.DEFINE_integer('batch_size', 128, 'batch size')
tf.app.flags.DEFINE_integer('hidden', 100, 'batch size')

# Global settings
szImg = 96
n_x = szImg * szImg
n_y = 30


def main(args=None):  # pylint: disable=unused-argument
    nn = SingleHiddenNN(
        input_shape=[FLAGS.batch_size, n_x],
        n_hidden=FLAGS.hidden,
        n_y=n_y)
    datasets = load_train_set(valid=FLAGS.valid)
    batches = BatchRenderer(
        datasets.train.images, 
        datasets.train.labels,
        FLAGS.batch_size)
    nn.train(
        batches, 
        datasets.valid, 
        lr=FLAGS.lr, 
        n_epoch=FLAGS.n_epoch)
    display_img_and_prediction(datasets.valid, nn, 4, 
        'imgs/model1-(fully-connected)-face.png')


def display_img_and_prediction(
    valid_set,
    nn,
    n=4, 
    oPngName='imgs/model1-(fully-connected)-face.png'):
    '''
    Plot the results
    '''
    N = n * n
    img = valid_set.images[:N]
    img = np.reshape(img, (N, -1))
    y = valid_set.labels[:N]
    p = nn.predict(img)
    
    plt.figure(figsize=(10, 10))
    for i in range(N):
        plt.subplot(n, n, i+1)
        pn = p[i]
        yn = y[i]
        plt.imshow(img[i].reshape((96, 96)), cmap='gray')
        plt.scatter(yn[0::2] * 48+48, yn[1::2] * 48+48,
        # plt.scatter(yn[0::2] * 96, yn[1::2] * 96,
            marker='o', edgecolors='r', facecolors='none')
        plt.scatter(pn[0::2] * 48+48, pn[1::2] * 48+48,
        # plt.scatter(pn[0::2] * 96, pn[1::2] * 96,
            marker='x', edgecolors='b')
        plt.axis('off')
        plt.subplots_adjust(
            left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
    
    plt.savefig(oPngName)


if __name__ == '__main__':
    tf.app.run()    # Run啥? 就是 run 本 script 中的 main() (TF根本莫名其妙)

    

Epoch 0000, 226.53145993, 218.66834408,  1.03595910
Epoch 0001, 0.16646391, 0.16469980,  1.01071107
Epoch 0002, 0.16365379, 0.16138339,  1.01406836
Epoch 0003, 0.15735120, 0.15493305,  1.01560769
Epoch 0004, 0.14952740, 0.14724015,  1.01553412
Epoch 0005, 0.14133042, 0.13906431,  1.01629541
Epoch 0006, 0.13299465, 0.13077066,  1.01700676
Epoch 0007, 0.12481390, 0.12255740,  1.01841179
Epoch 0008, 0.11672090, 0.11456048,  1.01885836
Epoch 0009, 0.10882099, 0.10685418,  1.01840647
Epoch 0010, 0.10153027, 0.09950666,  1.02033648
Epoch 0011, 0.09453835, 0.09253419,  1.02165851
Epoch 0012, 0.08796980, 0.08596051,  1.02337451
Epoch 0013, 0.08154885, 0.07979735,  1.02194940
Epoch 0014, 0.07585405, 0.07404049,  1.02449418
Epoch 0015, 0.07035331, 0.06868178,  1.02433741
Epoch 0016, 0.06556433, 0.06369928,  1.02927893
Epoch 0017, 0.06062266, 0.05907342,  1.02622563
Epoch 0018, 0.05631913, 0.05479542,  1.02780713
Epoch 0019, 0.05226400, 0.05084553,  1.02789755
Epoch 0020, 0.04860183, 0.04719448, 

SystemExit: 

TypeError: 'level' is an invalid keyword argument for this function

## 說明
```python
FLAGS = tf.app.flags.FLAGS  
tf.app.flags.DEFINE_float()
```
這段是在指定 main script 的 input arguments

然後我們先跳到最下面去看：  
```python
if __name__ == '__main__':
    tf.app.run()
```

我為了配合 TF 吃 input argument 的寫法，所以用 `tf.app.run` 來觸發 `main`  
這個寫法很讓人困惑：因為根本沒有哪個地方能看出來 `tf.app.run` 會觸發 `main` (但TF就是這樣 = = )  
這也是我很不喜歡 TF 的地方 (sigh~)


然後回到 `main`，這邊沒有什麼可講的，就是一般的步驟而已
1. 讀 data
2. 做成 bacthes (我是用 iterator 來做)
3. 宣告 model，開始 training

## Visualize the Computational Graph
這是 `tf.train.SummaryWriter` 的功用。
當我們創造這個物件時，它會在 `logdir` 中建立一些資料，以便 Tensorboard 使用。  
它可以存 image、audio、變數的 histogram，
但我們這邊只用到了它可以 visualize 整張 computational graph 的功能。
(所以後面都沒再繼續用它了)

當你執行過主程式之後。就可以在用下列指令發動 Tensorboard  
`tensorboard --logdir=[your_log_dir]`

然後可以到 localhost:6006 去看結果(graph 就像前面一章的圖那樣)。

## 小結
我不確定我是否有遺漏什麼部分，但能做到這些，基本上已經可以算是入了 Tensorflow 的門了。
資料夾裡應該還有 CNN 的範例 (model2)。  
但我要再過一段時間才能寫講解了。  
有問題再直接問我吧~